In [182]:
import pandas as pd
import numpy as np
from os.path import exists

# Team number need to be equal to number of room 
TEAMS = ["Team X", "Team Y", "Team Z"]
spots_pool = pd.read_csv("hints.csv")     

# TODO: 
# sort pool by min(diffictult to moving average diff)

# Store finish spot for later
finish_spot = spots_pool[spots_pool['room_name'].str.contains('~')].copy()

for team in TEAMS:
    spots_pool[team] = 0
    finish_spot[team] = len(spots_pool)

# Remove finish spot for now
spots_pool.drop(index=finish_spot.index.values.astype(int)[0], inplace=True)

while 0 in spots_pool[TEAMS].values:
    # Create 1 column per team to flag when spot is assigned
    for team in TEAMS:
        spots_pool[team] = 0
        
    for round_pick_counter in range(0, len(spots_pool)):
        rooms_picked_in_round = []

        for team_index, team in enumerate(TEAMS):
            current_team_max = spots_pool[team].max()

            avail_for_next_team = spots_pool[spots_pool['room_name'].isin(rooms_picked_in_round) == False]
            # return null sometimes
            possible_pick = avail_for_next_team[(avail_for_next_team[team] == 0) & (avail_for_next_team[team] != current_team_max + 1)]
            if possible_pick.empty == False:
                pick = possible_pick.sample(1)
            spot_picked_index = pick.index.values.astype(int)[0]
            spots_pool.loc[spots_pool.index == spot_picked_index, team] = current_team_max  + 1
            room_picked = pick['room_name'].values.astype(str)[0]
            rooms_picked_in_round.append(room_picked)

# Add back last spot
spots_pool = pd.concat([spots_pool,finish_spot], axis=0)

spots_pool['room_spot_ref'] = spots_pool['room_name'].map(str) + ' --> ' + spots_pool['spot_name'].map(str)

# Shift to display hiding spots 
for team in TEAMS:
    final_df = spots_pool.sort_values(by=team)[['difficulty_level', 'room_spot_ref', 'hint']]
    final_df.insert(0, 'team', team)
    final_df['hiding_spot'] = final_df['room_spot_ref'].shift(1, fill_value='Start Spot')
    final_df.insert(1, 'original_index', final_df.index)
    final_df = final_df.reset_index(drop=True)

    print(final_df)
    
    if not exists('game.csv'):
        final_df.to_csv('game.csv') 
    else:
        final_df.to_csv('game.csv', mode='a', header=False) 

      team  original_index  difficulty_level        room_spot_ref hint  \
0   Team X               4                 3    Room B --> Spot 1   H5   
1   Team X               5                 2    Room B --> Spot 2   H6   
2   Team X               7                 4    Room B --> Spot 4   H8   
3   Team X               6                 4    Room B --> Spot 3   H7   
4   Team X               1                 9    Room A --> Spot 2   H2   
5   Team X              11                 6    Room C --> Spot 4  H12   
6   Team X              10                 6    Room C --> Spot 3  H11   
7   Team X               9                 1    Room C --> Spot 2  H10   
8   Team X               2                 1    Room A --> Spot 3   H3   
9   Team X               8                 8    Room C --> Spot 1   H9   
10  Team X               0                 4    Room A --> Spot 1   H1   
11  Team X               3                 1    Room A --> Spot 4   H4   
12  Team X              12            